In [1]:
from transfer_learning_publication.data import CaravanDataSource
from transfer_learning_publication.transforms import PipelineBuilder, Log, ZScore

In [2]:
caravan = CaravanDataSource(
    base_path="/Users/cooper/Desktop/LSH_hive_data",
    region="camelsch"
)

basins = caravan.list_gauge_ids()

ts_data = caravan.get_timeseries(
    gauge_ids=[basins[0]],
    variables=["streamflow", "total_precipitation_sum"],
).collect()

In [3]:
static_data = caravan.get_static_attributes(
    gauge_ids=[basins[0]],
    columns=["area"],
).collect()

In [4]:
static_data

gauge_id,REGION_NAME,attribute_type,area
str,str,str,f64
"""camelsch_2009""","""camelsch""","""other""",5225.273653


In [ ]:
preprocessing_pipelines = (
    PipelineBuilder(group_identifier="gauge_id")
    .add_per_basin(Log(), columns=["streamflow"])
    .add_global(ZScore(), columns=["streamflow", "total_precipitation_sum"])
    .build()
)

data_transformed = preprocessing_pipelines.fit_transform(ts_data)
data_inv = preprocessing_pipelines.inverse_transform(data_transformed)

In [ ]:
ts_data.head(), data_transformed.head(), data_inv.head()